In [1]:
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import keras
from tensorflow.python.keras import regularizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from numpy import asarray
from tensorflow.keras.applications.nasnet import preprocess_input
import random
from keras import layers
from tensorflow.keras.utils import plot_model
from sklearn import model_selection
from tensorflow.keras.initializers import glorot_uniform

In [2]:
tf.config.list_physical_devices(
    device_type=None
)
path = "machine-learning-in-science-2022/"

In [3]:
def read_dataset(img_path,label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img,channels=3)
    img = tf.image.resize(img,(224, 224))
    img = tf.cast(img, tf.float32)
    return img,label

In [4]:
def add_noise(img):
    noise = tf.random.normal(tf.shape(img),0,5)
    img += noise
    img = tf.clip_by_value(img, 0, 255)
    return img

In [19]:
df = pd.read_csv(path + 'signal_detection.csv')
img_path = []
signal = []
for _,row in df.iterrows():
    img_path.append(path + 'training_data/training_data_added/training_data/' + str(int(row["image_id"])) + '.png')
    signal.append(row["signal"])

In [20]:
for i in range(len(signal)):
    if signal[i] == 'red':
        signal[i] = 2
    elif signal[i] == 'green':
        signal[i] = 1
    else:
        signal[i] = 0         
signal = np.array(signal)

In [29]:
img_path_train,img_path_val,signal_train,signal_val = model_selection.train_test_split(img_path,signal, test_size=0.2, train_size=0.8, random_state=None, shuffle=True, stratify=signal)

In [30]:
signal_one_hot_train = np.zeros((signal_train.size, signal_train.max()+1))
signal_one_hot_train[np.arange(signal_train.size),signal_train] = 1

signal_one_hot_val = np.zeros((signal_val.size, signal_val.max()+1))
signal_one_hot_val[np.arange(signal_val.size),signal_val] = 1

In [31]:
ds_train_SP = tf.data.Dataset.from_tensor_slices((img_path_train, signal_one_hot_train))
ds_train_SP = ds_train_SP.map(read_dataset).map(lambda image, label: (tf.image.random_contrast(image, lower = 0.9, upper = 1), label)
                                                                ).map(lambda image, label: (tf.image.random_saturation(image, lower = 0.8, upper = 1.2),label)
                                                                     ).map(lambda image, label: (tf.image.random_brightness(image, 0.5 ,1.5),label)
                                                                          ).map(lambda image, label: (add_noise(image), label)
                                                                               ).batch(128).repeat(4)
                                                                                   

ds_val_SP = tf.data.Dataset.from_tensor_slices((img_path_val, signal_one_hot_val))
ds_val_SP = ds_val_SP.map(read_dataset).batch(128)
                      

In [50]:
mobile = tf.keras.applications.MobileNetV3Large(input_shape=[224,224,3])    
x = mobile.layers[-10].output
x = Dropout(0.2)(x)
x = Conv2D(2,(1,1),strides = (1, 1),kernel_initializer = glorot_uniform(seed=0))(x)
x = Flatten()(x)
signal1 = Dense(64, activation='relu', name='signal_hidden')(x)
signal_pred = Dense(3, activation='sigmoid', name='signal')(signal1)
model = Model(inputs = mobile.input, outputs = [signal_pred])

for layer in model.layers[:-44]:
    layer.trainable = False 

model.compile(optimizer="adam", loss={"signal": "categorical_crossentropy"}, metrics = "accuracy")

In [51]:
model.fit(ds_train_SP, epochs=10, validation_data=ds_val_SP)

Epoch 1/10


ValueError: in user code:

    File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/engine/training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-p

In [35]:
model.save("signal")

2022-05-11 11:03:32.545910: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: signal/assets


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [38]:
def read_dataset_test(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img,channels=3)
    img = tf.image.resize(img,(224, 224))
    img = tf.reshape(img,(1,224,224,3))
    img = tf.cast(img, tf.float32)
    return img

In [49]:
model.predict(read_dataset_test(path + 'test_data/test_data/955.png'))

array([[1.3999114e-05, 9.6282369e-01, 9.9706370e-01]], dtype=float32)

In [58]:
import os
import os
os.getcwd()
collection = "machine-learning-in-science-2022/images_with_curated_names/"
filenames = []
for i, filename in enumerate(os.listdir(collection)):
    if filename!= '.DS_Store':
        filenames.append(int(filename[:-4]))
filenames = sorted(filenames)

In [59]:
k = 13799
for i in filenames:
    os.rename("machine-learning-in-science-2022/images_with_curated_names/" + str(i) + ".png", "machine-learning-in-science-2022/images_with_curated_names/" + str(k) + ".png")
    k = k+1

In [57]:
for i in filenames:
    os.rename("machine-learning-in-science-2022/images_with_curated_names/" + str(i) + ".png", "machine-learning-in-science-2022/images_with_curated_names/" + str(i-13798) + ".png")